**Pacotes**

In [179]:
#%pip install transformers
#%pip install googletrans==3.1.0a0
#%pip install spacy
#!python -m spacy download en_core_web_lg 

In [180]:
from googletrans import Translator
from transformers import pipeline
import textwrap
from transformers import BertTokenizer, BertModel
from sklearn.cluster import SpectralClustering
import torch
import spacy
import numpy as np
from collections import Counter
nlp = spacy.load("en_core_web_lg")

**DADOS**

In [181]:
#avaliacoes
avaliacoes_pt = [
"Estou extremamente satisfeito com o atendimento prestado pela equipe.",
"Os benefícios do meu cartão de crédito são bons.",
"Limite muito baixo no cartão de crédito. Também não gostei dos benefícios e da taxa de juros",
"O atendimento e suporte ao cliente é excepcional.",
"A taxa de juros do meu cartão de crédito é excessivamente alta.",
"Atendimento foi demorado e burocrático. Não atendem o telefone",
"O aplicativo do cartão de crédito é confuso.",
"O atendimento telefônico nem sempre está disponível, e quando está, as filas de espera são longas.",
"A variedade de benefícios oferecidos pelo meu cartão de crédito é baixa.",
"O atendimento ao cliente é ruim, falta suporte adequado"
]

In [182]:
texto_pt = "O cantor João Carreiro morreu nesta quarta-feira (3), informou a assessoria do artista. Ele tinha 41 anos. O sertanejo não resistiu a uma cirurgia para colocar uma válvula no coração.\
A mulher de João, Francine Caroline, postou um vídeo nas redes sociais durante a manhã desta quarta, afirmando que o cantor estava no centro cirúrgico.\
Durante a tarde, Francine publicou uma nova mensagem, dizendo que o coração de João já estava funcionando sozinho com a nova válvula e que a cirurgia estava sendo finalizada.\
Horas depois, Francine postou um texto pedindo orações pelo cantor.\
Natural de Cuiabá, o sertanejo ficou nacionalmente conhecido por participar da dupla João Carreiro & Capataz, que fez muito sucesso durante a década de 2000.\
Em 2009, a música Bruto, Rústico e Sistemático fez parte da trilha sonora da novela Paraíso, da TV Globo.\
Em 2014, a dupla se separou, e João Carreiro seguiu carreira solo. Em dezembro de 2023, o cantor lançou as gravações de duas músicas: Meu Avô e A Coroa É Meu Chapéu.\
Na virada do ano, João Carreiro se apresentou na cidade de Pedra Preta (MT)."

#https://g1.globo.com/pop-arte/noticia/2024/01/03/morre-cantor-joao-carreiro.ghtml

A notícia tratada foi a que noticiou sobre a morte de João Carreiro, o qual particularmente gostava bastante, e infelizmente faleceu no dia 03/01/2023. João carreiro foi um grande músico sertanejo, compositor e cantor, ganhou destaque na música sertaneja, sendo muito querido por amantes da música sertaneja.

**Funções**

In [183]:
def Exibir(texto):
  
  texto = textwrap.wrap(texto, width=135)
  #Imprime cada linha do resumo
  for linha in texto:
    print(linha)

In [184]:
def tradutor(texto,origem, destino):

  translator = Translator() 
  texto_final = translator.translate(texto, dest=str(destino),src=str(origem)).text

  return texto_final

**Sentiment Analysis**

Sentiment Analysis tries to identify and categorize opinions in text, often marked as positive and negative (sometimes neutral as well).

In [185]:
#Processar cada comentário
novalista = []
for texto in avaliacoes_pt:
    #traduzir
    texto = tradutor(texto,"pt","en")
    #Converter para minúsculas
    texto = texto.lower()
    doc = nlp(texto)
    #lematizar
    novotexto = ' '.join([token.lemma_ for token in doc])
    novalista.append(novotexto)

avaliacoes_en = novalista

In [186]:
classifier = pipeline("sentiment-analysis")

resultado = classifier(avaliacoes_en)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [187]:
for i, j in zip(resultado, avaliacoes_pt):
    print("Avaliação:",j)
    print("Sentimento:",i['label'])
    print()

Avaliação: Estou extremamente satisfeito com o atendimento prestado pela equipe.
Sentimento: POSITIVE

Avaliação: Os benefícios do meu cartão de crédito são bons.
Sentimento: POSITIVE

Avaliação: Limite muito baixo no cartão de crédito. Também não gostei dos benefícios e da taxa de juros
Sentimento: NEGATIVE

Avaliação: O atendimento e suporte ao cliente é excepcional.
Sentimento: POSITIVE

Avaliação: A taxa de juros do meu cartão de crédito é excessivamente alta.
Sentimento: NEGATIVE

Avaliação: Atendimento foi demorado e burocrático. Não atendem o telefone
Sentimento: NEGATIVE

Avaliação: O aplicativo do cartão de crédito é confuso.
Sentimento: NEGATIVE

Avaliação: O atendimento telefônico nem sempre está disponível, e quando está, as filas de espera são longas.
Sentimento: NEGATIVE

Avaliação: A variedade de benefícios oferecidos pelo meu cartão de crédito é baixa.
Sentimento: NEGATIVE

Avaliação: O atendimento ao cliente é ruim, falta suporte adequado
Sentimento: NEGATIVE



**QUESTION ANSWERING**

In [188]:
#tratamento de dados, noticia
texto_en = tradutor(texto_pt,"pt","en")

In [235]:
question_answerer = pipeline("question-answering")

def question(question_pt, texto):
    question = tradutor(question_pt,"pt","en")
    resposta = question_answerer( question=str(question), context=texto)['answer']
    resposta = tradutor(resposta,"en","pt")
    return print(question_pt,"\n",resposta)

#teste
question("O que aconteceu?", texto_en)
question("Quando João morreu?", texto_en)
question("Como João Carreiro morreu?", texto_en)
question("Onde estava João na virada do ano?", texto_en)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


O que aconteceu? 
 Morreu o cantor João Carreiro
Quando João morreu? 
 Quarta-feira (3)
Como João Carreiro morreu? 
 cirurgia para colocar uma válvula
Onde estava João na virada do ano? 
 Pedra Preta (MT)


**SUMARIZADOR**

In [190]:
summarizer = pipeline("summarization")

def resumir(texto):

  resumo = summarizer(texto_en)
  resumo = resumo[0]['summary_text']
  resumo = tradutor(resumo,"en","pt")
  
  return resumo

resumo = resumir(texto_en)
Exibir(resumo)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


João Carreiro morreu nesta quarta-feira (3), segundo o assessor do artista. O cantor country não sobreviveu à cirurgia para colocar uma
válvula no coração. A esposa de João, Francine Caroline, postou um vídeo nas redes sociais na manhã desta quarta-feira, informando que
o cantor estava no centro cirúrgico. Em 2009, a música Bruto, Rústico e Sistemático fez parte da trilha sonora da novela Paraíso.


**CLUSTERING**

In [191]:
#Carregar modelo BERT pré-treinado e tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [192]:
#Processar cada comentário
nova_lista = []
for texto in avaliacoes_en:
    #spacy format
    doc = nlp(texto)
    #selecionar apenas substantivos
    novo_texto = ' '.join([token.lemma_ for token in doc if token.pos_ == 'NOUN'])
    nova_lista.append(novo_texto)

avaliacoes_cluster = nova_lista

In [193]:
#Converter a lista de textos em uma única string para processamento de frequência
all_texts = ' '.join(avaliacoes_cluster)

#Processamento de frequência das palavras
word_frequencies = Counter(all_texts.split())

#Calcular a porcentagem de frequência de cada palavra
total_documents = len(avaliacoes_cluster)
word_percentages = {word: freq / total_documents for word, freq in word_frequencies.items()}

#Filtrar palavras que aparecem mais de 50% das vezes ou menos que 10%
filtered_words = [word for word, percentage in word_percentages.items() if percentage > 0.50 or percentage < 0.10]

#Remover palavras filtradas dos documentos
texto_final = [' '.join([word for word in texto.split() if word not in filtered_words]) for texto in avaliacoes_cluster]

In [206]:
#Tokenização e codificação dos textos
max_length = 200
input_ids = [tokenizer.encode(texto_final, add_special_tokens=True, max_length=max_length, truncation=True) for texto in texto_final]

#Preencher ou truncar as sequências para ter o mesmo tamanho
input_ids = [seq + [0] * (max_length - len(seq)) for seq in input_ids]

#Obter embeddings do BERT
with torch.no_grad():
    outputs = model(torch.tensor(input_ids))

#Extrair embeddings da camada oculta final
last_hidden_states = outputs.last_hidden_state

#Somar os embeddings de todas as palavras para obter os embeddings dos documentos
document_embeddings = last_hidden_states.mean(dim=1).numpy()
spectral = SpectralClustering(n_clusters=2, affinity='rbf',gamma=0.0001,random_state=40)
clusters = spectral.fit_predict(document_embeddings)

#Imprimir os textos em cada cluster
for cluster_id in np.unique(clusters):
    print(f"Cluster {cluster_id}:")
    for i, texto in enumerate(texto_final):
        if clusters[i] == cluster_id:
            print(" - ",tradutor(texto,"en","pt"))

Cluster 0:
 -  limitar a taxa de juros do benefício do cartão de crédito
 -  taxa de juros do cartão de crédito
 -  solicitação de cartão de crédito
 -  fila de atendimento telefônico
 -  faixa benefício oferta cartão de crédito
 -  Serviço de atendimento ao consumidor
Cluster 1:
 -  equipe de prestação de serviços
 -  benefício do cartão de crédito
 -  Serviço de atendimento ao consumidor
 -  telefone de serviço
